In [1]:
_RIOT_API_KEY='RGAPI-eb715053-5bb4-49af-b624-970e0673d75f'
from riotwatcher import LolWatcher
import helper_functions as help
import constants
import pandas as pd
from datetime import datetime
import mysql.connector
from sqlalchemy import create_engine
import concurrent.futures

lol_watcher = LolWatcher(_RIOT_API_KEY)

1. Implement retry for when the URL gets too many requests for 1 match. probably as simple as a wrapper function around the slurps in slurp_data that retries if the clienter error exception is returned
e.g. Exception 429 Client Error: Too Many Requests for url: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4708472978/timeline thrown. Cancelling futures.

2. Handle this exception: Exception 404 Client Error: Not Found for url: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_3891774034 thrown. Cancelling futures.

3. Handling processing by puuid instead of summonerName? 
    GoofyGriefer	cT70cPTfqJ_gLJrZRqSTBO0lO3ZfM25tP4MRfvVsPw5CF5EHcdPILgojfnvBMudR9vJmnNAtFxiBvg
    CerebralPlayer	cT70cPTfqJ_gLJrZRqSTBO0lO3ZfM25tP4MRfvVsPw5CF5EHcdPILgojfnvBMudR9vJmnNAtFxiBvg
    PepegaMindset	cT70cPTfqJ_gLJrZRqSTBO0lO3ZfM25tP4MRfvVsPw5CF5EHcdPILgojfnvBMudR9vJmnNAtFxiBvg

In [3]:
engine = help.create_mysql_engine()

In [15]:
# ~25 seconds for 50 matches. More if we've hit our API Limit.
# 8m 16s for 800 records
# PepegaMindset
# CerebralPlayer
help.collect_riot_api_data('Boss 0f This Gym', lol_watcher, 'lol_match_timeline',
                            engine, match_upload_limit=800, num_worker_threads=2)

0 matches to slurp!


In [16]:
# 23.7 seconds for 50 matches, 1 thread
# 7.4 seconds for 20 matches, 2 threads
# 5 seconds for 20 matches, 4 threads
# 4.7 seconds for 20 matches, 8 threads
# 5.8 seconds for 50 matches, 8 threads

help.collect_riot_api_data('Boss 0f This Gym', lol_watcher, 'lol_match_details',
                            engine, match_upload_limit=800, num_worker_threads=2)


27 matches to slurp!
Uploaded match id NA1_4630769530 lol_match_details data.
Uploaded match id NA1_4630762225 lol_match_details data.
Uploaded match id NA1_4566932094 lol_match_details data.
Uploaded match id NA1_4566902372 lol_match_details data.
Uploaded match id NA1_4556569779 lol_match_details data.
Uploaded match id NA1_4556358077 lol_match_details data.
Uploaded match id NA1_4556099487 lol_match_details data.Uploaded match id NA1_4554378852 lol_match_details data.

Uploaded match id NA1_4550042149 lol_match_details data.
Uploaded match id NA1_4550050576 lol_match_details data.
Uploaded match id NA1_4490916049 lol_match_details data.
Uploaded match id NA1_4547723290 lol_match_details data.
Uploaded match id NA1_4490836221 lol_match_details data.
Uploaded match id NA1_4490784654 lol_match_details data.
Uploaded match id NA1_4490608309 lol_match_details data.
Uploaded match id NA1_4490751690 lol_match_details data.
Uploaded match id NA1_4327930754 lol_match_details data.
Uploaded m